In [1]:
from fastai.vision.all import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callback.tracker import SaveModelCallback
import argparse

from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy
import torchvision.transforms as T
import torch
from torch.nn.parallel import DistributedDataParallel
from torchvision.transforms.functional import *

from PIL import Image
import requests

import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn

import argparse
from models.utils.ARViT2D import ARViT2D
from models.utils.distance_loss import ARViT2D_Loss
from models.utils.new_losses import *
from models.utils.metrics import *
from models.utils.dataLoader import *
from models.utils.datasets import *

In [10]:
torch.cuda.set_device("cuda:0")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
H = 256
W= 256
bs = 80

In [12]:
#model = ARViT2D(num_encoder_layers = 6, nhead=8, num_classes = 4, mask=None, pos_enc = "sin", 
#                batch_size=bs, in_chans=3, hidden_dim=512, image_h=256, image_w=256, grid_l=16, 
#                gm_patch = 32, use_patches=True, attn_layer = 1,penalty_factor="2", alpha=4, 
#                beta=500, gamma=0.1)

 #model_dir = Path.home()/'Luiz/saved_models/AROB'
# weights_dir = model_dir/'best/ARViT2D-Base-12L.pth'
# weights = torch.load(weights_dir)
# model.pos = nn.Parameter(weights['pos'],requires_grad=False)
# model.load_state_dict(weights,strict=False)
# model = model.eval()

In [13]:
model_dir = Path.home()/'Luiz/saved_models/AROB'
net = load_learner(model_dir/'ARViT2D-Base-6L.pkl', cpu=False)
weights_dir = model_dir/'best/ARViT2D-L2.pth'
model = net.model
weights_dict = load_learner(weights_dir, cpu=False)
model.load_state_dict(weights_dict)
model = model.eval()

In [6]:
def paths(dataset = 'IMAGENETTE'): 
    if dataset == 'IMAGENETTE':
        data_path = untar_data(URLs.IMAGENETTE)
    elif dataset == 'IMAGEWOOF':
        data_path = untar_data(URLs.IMAGEWOOF)
    elif dataset == 'CIFAR':
        data_path = untar_data(URLs.CIFAR)
    elif dataset == 'CIFAR_100':
        data_path = untar_data(URLs.CIFAR_100)  
    elif dataset == 'CALTECH_101':    
        data_path = untar_data(URLs.CALTECH_101) 
    elif dataset == 'FLOWERS':
        data_path = untar_data(URLs.FLOWERS)
    else:
        data_path = None
        
    return data_path

In [7]:
def load_model(model, layer='L3', n_classes=10,dataset='IMAGENETTE'):

    model.head = nn.Linear(516*16*16, n_classes)
    model_dir = Path.home()/'Luiz/saved_models/AROB/finetuned'
    #net = load_learner(model_dir/'ARViT2D-Base_6layers.pkl', cpu=False)
    name = 'ARViT2D-'+'L2'+'_'+dataset+'.pth'
    weights_dir = model_dir/name
    weights = torch.load(weights_dir)
    model.pos = nn.Parameter(weights['pos'],requires_grad=False)
    #model = net.model
    model.load_state_dict(weights)
    #weights_dict = load_learner(weights_dir, cpu=False)
    #model.load_state_dict(weights_dict)
    model = model.eval()
        
    return model
            
def data_loader(path):
    transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                     get_items=get_image_files, 
                     splitter=RandomSplitter(),
                     get_y=parent_label,
                     item_tfms=Resize(H,W),
                     batch_tfms=transform)

    dloader = data.dataloaders(path,bs=bs)
    return dloader

def flowers_loader(path):
    Path.BASE_PATH = path
    path.ls()
    df = pd.read_csv('data/flowers.csv')

    transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def get_x(r): return path/r['name']
    def get_y(r): return r['class']

    dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                       n_inp=1,
                       splitter=RandomSplitter(seed=42),
                       get_x= get_x,
                       get_y= get_y, 
                       item_tfms = Resize(256),
                       #batch_tfms= Normalize.from_stats(*imagenet_stats)
                       batch_tfms= transform
                      )

    dloader = dblock.dataloaders(df, bs=bs)
    return dloader

In [8]:
def model_test(model, n_class, layer='L3' ,dataset='IMAGENETTE', epochs=1, lr=1e-9):
    path_data = paths(dataset)
    #print(path_model, weights_path, path_data)
    model = load_model(model, layer=layer, n_classes=n_class,dataset=dataset)
        
    if dataset == 'FLOWERS':
        dloader = flowers_loader(path_data)
    else:
        dloader = data_loader(path_data)
    #Defining the Loss Function
    critic_loss = SingleLabelCriticLoss()
    
    #Wraping the Learner
    learner = Learner(dloader, model, loss_func=critic_loss, metrics=[Accuracy])
    learner.fit_one_cycle(epochs, lr)
    #learner.fine_tune(60, base_lr=5e-4, freeze_epochs=30)

In [12]:
model_test(model, n_class=10, layer='L2' ,dataset='CIFAR')

epoch,train_loss,valid_loss,Accuracy,time
0,0.588430,0.527991,0.817833,03:03


In [14]:
model_test(model, n_class=100, layer='L2' ,dataset='CIFAR_100')

epoch,train_loss,valid_loss,Accuracy,time
0,1.342447,1.128662,0.699167,03:04


In [15]:
model_test(model, n_class=102, layer='L2' ,dataset='FLOWERS')

epoch,train_loss,valid_loss,Accuracy,time


RuntimeError: CUDA out of memory. Tried to allocate 240.00 MiB (GPU 0; 10.76 GiB total capacity; 8.78 GiB already allocated; 88.31 MiB free; 9.65 GiB reserved in total by PyTorch)

In [ ]:
model_test(model, n_class=10, layer='L2' ,dataset='IMAGENETTE')

In [ ]:
model_test(model, n_class=10, layer='L2' ,dataset='IMAGEWOOF')

In [14]:
model_test(n_class=100, name='ARViT-L1-G32', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.939998,0.576333,0.848500,03:58


In [15]:
model_test(n_class=100, name='ARViT-L1-G32_Best', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.942338,0.605219,0.840917,03:57


In [16]:
model_test(n_class=100, name='ARViT-L6-G32', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.951208,0.593923,0.847417,03:58


In [17]:
model_test(n_class=100, name='ARViT-L6-G32_Best', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.913626,0.584167,0.848000,03:58


In [18]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.287862,0.170025,0.948096,00:52


In [19]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.238245,0.161538,0.951830,00:52


In [20]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.300479,0.175379,0.945108,00:52


In [21]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.269931,0.184755,0.944735,00:52


In [22]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.479238,0.397447,0.861167,03:38


In [23]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.543407,0.385890,0.869750,03:43


In [24]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.526396,0.372448,0.871000,03:39


In [25]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.468442,0.369256,0.874000,03:40


In [26]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.825701,0.611274,0.802703,00:50


In [27]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.808609,0.618858,0.801158,00:51


In [28]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.733986,0.617505,0.799614,00:50


In [29]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.785065,0.599061,0.808108,00:51


In [13]:
def paths2(layer = 1, dataset = 'IMAGENETTE', base_model=False, best_model=False): 
    model_dir = Path.home()/'Luiz/saved_models'
    if base_model == True:
        model_path = model_dir/'GramImageNet_Rotation_16x16grid_epochs-90_BaseModel.pkl'
    else:
        file_name = 'GramImageNet_Rotation_16x16grid_epochs-90-beta-5e-6_PenaltyFactor3_Layer'+str(layer)+'.pkl'
        model_path = model_dir/file_name
        
    if best_model == True:
        weight_dir = Path.home()/'Luiz/gan_attention/models/pretrained/'
        if base_model == True:
            file_name = 'Best_Model_BaseModel_ImageNet_Rotation_16x16grid.pth'
        else:
            file_name = 'Best_Model_Layer'+str(layer)+'_ImageNet_Rotation_16x16grid.pth'
        weights_path = weight_dir/file_name
    else:
        weights_path = None
    
    if dataset == 'IMAGENETTE':
        data_path = untar_data(URLs.IMAGENETTE)
    elif dataset == 'IMAGEWOOF':
        data_path = untar_data(URLs.IMAGEWOOF)
    elif dataset == 'CIFAR':
        data_path = untar_data(URLs.CIFAR)
    elif dataset == 'CIFAR_100':
        data_path = untar_data(URLs.CIFAR_100)    
    elif dataset == 'CALTECH_101':    
        data_path = untar_data(URLs.CALTECH_101) 
    else:
        data_path = None
        
    return model_path, weights_path, data_path

In [23]:
def data_loader(path):
    transform = ([*aug_transforms()])#,Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                     get_items=get_image_files, 
                     splitter=RandomSplitter(),
                     get_y=parent_label,
                     item_tfms=Resize(H,W),
                     batch_tfms=transform)

    dloader = data.dataloaders(path,bs=bs)
    return dloader

In [16]:
def fine_tune_model(n_class, layer = 1, dataset = 'IMAGENETTE', base_model=False, best_model=False, epochs=[1,1], lr=5e-4):
    path_model, weights_path, path_data = paths2(layer, dataset, base_model, best_model)
    print(path_model, weights_path, path_data)
    print("Training on", dataset)
    model = load_model(path_model, weights_path)
    model_head(model, n_class)
        
    dloader = data_loader(path_data)
    #Defining the Loss Function
    critic_loss = SingleLabelCriticLoss()  

    #Wraping the Learner
    learner = Learner(dloader, model, loss_func=critic_loss, metrics=[_Accuracy])
    #learner.fit_one_cycle(50, 0.002)
    learner.fine_tune(epochs[0], base_lr=lr, freeze_epochs=epochs[1])

In [68]:
data_path = untar_data(URLs.FOOD)

In [69]:
(data_path).ls()

(#8) [Path('/home/atsumilab/.fastai/data/food-101/train.json'),Path('/home/atsumilab/.fastai/data/food-101/test.json'),Path('/home/atsumilab/.fastai/data/food-101/images'),Path('/home/atsumilab/.fastai/data/food-101/labels.txt'),Path('/home/atsumilab/.fastai/data/food-101/test.txt'),Path('/home/atsumilab/.fastai/data/food-101/h5'),Path('/home/atsumilab/.fastai/data/food-101/train.txt'),Path('/home/atsumilab/.fastai/data/food-101/classes.txt')]

In [66]:
transform = ([*aug_transforms()])#,Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=using_attr(RegexLabeller(r'(?<=image_)(.*)(?=.jpg)'),'name'),
                 item_tfms=Resize(256,256),
                 batch_tfms=transform)

dloader = data.dataloaders(data_path/"jpg", bs=5)

Could not do one pass in your dataloader, there is something wrong in it


In [67]:
files = get_image_files(data_path/"jpg")
len(files)

8189

'009'